In [1]:
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
colors = px.colors.qualitative.Plotly

import pandas as pd
pd.set_option('display.max_rows', 100)

import numpy as np
from scipy import stats

import multiprocessing as mp

from ARCH import basic, modelling, plot

# Synonymous mutations

In [14]:
# Dataset
df = pd.read_excel(r'LBC_ARCHER.synonymous.Aug_2020.xlsx')
syn, syn_grad = basic.load(df)

In [47]:
# Histogram
fig = px.histogram(df[df.AF<0.05], x="AF", histnorm='probability')
fig.update_layout(title='VAF distribution of synonymous mutations')
fig.update_yaxes(title='probability')
fig.update_xaxes(title='VAF')
fig.write_image('plots/Synonymous_mutations_histogram.png')
fig

In [48]:
melt_syn = basic.melt(syn, filter=0.07)
melt_syn = melt_syn[melt_syn['regularized_gradient'].notnull()]

In [49]:
fig = go.Figure()
for part in syn:
    for traj in part.trajectories:
        fig.add_trace(go.Scatter(x=traj.data.wave, y=traj.data.AF, name=traj.mutation))
fig.update_yaxes(title='VAF')
fig.update_xaxes(title='years since wave 1')
fig

In [50]:
melt_syn['bin'] = pd.cut(melt_syn['AF'], 50)

fig = go.Figure()
for bin in melt_syn.bin.unique():
    fig.add_trace(go.Scatter(x=melt_syn[melt_syn['bin']==bin].AF, y=melt_syn[melt_syn['bin']==bin].regularized_gradient,mode='markers'))

fig.update_layout(title='Noise distribution')
fig.update_xaxes(title='VAF')
fig.update_yaxes(title='Regularized Difference')

In [51]:
syn_normal_dist =[]  # List of tuples containing bin center, computed standard deviation, and associated figure
left_limit = 0.007
right_limit = 0.02
for i, bin in enumerate(melt_syn.bin.unique()):
    j = np.mean([bin.left,bin.right])
    if left_limit< j <right_limit:
        df = melt_syn[melt_syn['bin']== bin]
        out, fig = modelling.gauss_fit(df.regularized_gradient)#, center=0)
        syn_normal_dist.append((j, out.params['g1_center'].value, out.params['g1_sigma'].value, fig))

In [52]:
i=1
print(f'AF center : {syn_normal_dist[i][0]}')
print(f'standard deviation {syn_normal_dist[i][1]}')
syn_normal_dist[i][2]

AF center : 0.009345
standard deviation 0.00024206376522412893


0.0018282926187347392

In [67]:
# Scatter plot of AF ~ fitted standard deviations

AF = [col[0] for col in syn_normal_dist]
sigma = [col[2] for col in syn_normal_dist]

# Fit a line going through the origin
af_array = np.asarray(AF)
af_array = af_array[:,np.newaxis]
slope, _, _, _ = np.linalg.lstsq(af_array,np.power(sigma,2),rcond=None)

# Scatter plot
x= np.linspace(0,right_limit,2)
y= x*slope

fig=go.Figure()
fig.add_trace(go.Scatter(x=AF,y=np.power(sigma,2),mode='markers',name='data'))
fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name='linear fit'))
fig.update_layout(title=f'Variance of gradients ~ VAF with slope {round(slope[0],5)}')
fig.update_xaxes(title='VAF (binned)')
fig.update_yaxes(title='Variance')
fig.write_image('Results/Figures/Filtering_and_Fitting/Synonymous_mutations_variance_vs_VAF.png',width=800,height=600)
fig

In [58]:
HSC = 0.1 / slope
HSC

array([281.83560078])

 # Non synonymous mutations

In [54]:
df = pd.read_excel(r'LBC_ARCHER.non-synonymous.Aug_2020.xlsx')
lbc, lbc_grad = basic.load(df)

melt_lbc = basic.melt(lbc, filter=0.02)
melt_lbc = melt_lbc[melt_lbc['gradient']<0.02]
melt_lbc = melt_lbc[melt_lbc['regularized_gradient'].notnull()]

In [55]:
melt_lbc['bin'] = pd.cut(melt_lbc['AF'], 50)

fig = go.Figure()
for bin in melt_lbc.bin.unique():
    fig.add_trace(go.Scatter(x=melt_lbc[melt_lbc['bin']==bin].AF,
                             y=melt_lbc[melt_lbc['bin']==bin].regularized_gradient,
                             mode='markers'))

fig.update_layout(title='Fluctuations distribution')
fig.update_xaxes(title='VAF')
fig.update_yaxes(title='Regularized differences')
fig.show()
fig.write_image('Results/Figures/Filtering_and_Fitting/Gradient_distribution.png',width=1200,height=600)

In [56]:
lbc_normal_dist =[]  # List of tuples containing bin center, computed standard deviation, and associated figure

left_limit = 0.01
right_limit = 0.019
for i, bin in enumerate(melt_lbc.bin.unique()):
    j = np.mean([bin.left,bin.right])
    if left_limit< j <right_limit:
        df = melt_lbc[melt_lbc['bin']== bin]
        out, fig = modelling.gauss_fit(df.regularized_gradient,
                                       center=None)  # you can force the center of the distribution to be any float
        lbc_normal_dist.append((j, out.params['g1_center'].value, out.params['g1_sigma'].value, fig))

In [68]:
# Scatter plot of AF ~ fitted standard deviations

AF = [col[0] for col in lbc_normal_dist]
sigma = [col[2] for col in lbc_normal_dist]

# Create data to plot fitted line
af_array = np.asarray(AF)
af_array = af_array[:,np.newaxis]
slope, _, _, _ = np.linalg.lstsq(af_array,np.power(sigma,2),rcond=None)

x = np.linspace(0, right_limit, 2)
y = slope*x

# Scatter plot
fig=go.Figure()
fig.add_trace(go.Scatter(x=AF,y=np.power(sigma,2),mode='markers',name='data'))
fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name=f'linear fit'))
fig.update_layout(title=f'Variance of gradients ~ VAF with slope {round(slope[0],5)}')
fig.update_xaxes(title='VAF of bin')
fig.update_yaxes(title='Variance')
fig.write_image('Results/Figures/Filtering_and_Fitting/Nonsynonymous_mutations_variance_vs_VAF.png',width=800,height=600)
fig

In [37]:
HSC = 0.1 / slope
HSC

array([228.4517361])

In [38]:
# Compute the standard deviation of gradients
x = np.linspace(0,0.02,100)
std = np.sqrt(slope*x)

fig = go.Figure()
fig.add_trace(go.Scatter(x=melt_lbc.AF,
                         y=melt_lbc.regularized_gradient.abs(),
                         mode='markers',
                         name = 'Data points'))
fig.add_trace(go.Scatter(x=x,
                         y=2*std,
                         mode='lines',
                         name='2 Standard deviations'))

fig.update_layout(title='Fluctuations distribution')
fig.update_xaxes(title='AF')
fig.update_yaxes(title='Regularized differences')
fig

# Filtering and modelling trajectories

In [39]:
df = pd.read_excel(r'LBC_ARCHER.non-synonymous.Aug_2020.xlsx')
lbc, lbc_grad = basic.load(df)

slope = 0.00043773
HSC = 0.1 / slope
initial_pop = 1/HSC  # VAF corresponding to 1 HSC

## Filtering 

In [40]:
# Compute the standard deviation of absolute regularized total gradients
x = np.linspace(0,0.5,1000)
std = np.sqrt(slope*x)

fig = go.Figure()
traj = lbc[0].trajectories[0]
fig.add_trace(go.Scatter(x=[traj.data.AF.iloc[0]],
                         y=[np.abs(traj.gradient)],
                         line=dict(color=colors[0]),
                         mode = 'markers',
                         legendgroup="group",
                         name="trajectories"
                         ))
for part in lbc:
    for traj in part.trajectories:
        if traj.data.AF.mean() < 0.5  and \
           len(traj.data)>2 :
            if np.abs(traj.gradient) > 2*np.sqrt(slope*traj.data.AF.iloc[0]):
                # color coding differently trajectories used to fit
                fig.add_trace(go.Scatter(x=[traj.data.AF.iloc[0]],
                              y=[np.abs(traj.gradient)],
                              line=dict(color=colors[0]),
                              legendgroup="group",
                              showlegend=False,
                              hovertemplate = f'Mutation: {traj.mutation}'
                              ))
            else :
                fig.add_trace(go.Scatter(x=[traj.data.AF.iloc[0]],
                              y=[np.abs(traj.gradient)],
                              line=dict(color=colors[2]),
                              legendgroup="group",
                              showlegend=False,
                              hovertemplate = f'Mutation: {traj.mutation}'
                              ))
fig.add_trace(go.Scatter(x=x, y=2*std, name='2 Standard deviations',line=dict(color=colors[1])))
fig.update_layout(title='Trajectory filtering')
fig.update_xaxes(title='AF')
fig.update_yaxes(title='Regularized gradient')
fig.write_image('Results/Figures/Filtering_and_Fitting/Filtering.png',width=1000,height=600)
fig

Sample LBC360453 seems to have some wrong measurements in the first time point. This translates in the cluster around 0.4 in the above scatter-plot.

In [41]:
plot.plot_id(syn,'LBC360453',germline=True)

# Modelling

We model all trajectories above the red line with a logistic curve

In [42]:
"""" Modelling all trajectories filtered by VAF and gradient
A trajectory 'traj' is selected for modelling if:
- 'traj' has more than 3 data points 
- 'traj.gradient' grows more than 2 standard deviations of expected distribution of noise.
- 'traj.AF.mean()' is within the range (0.005, 0.5) to get rid of sequenecing errors and 
   mutations altering dna copies
"""""

# initialize a list of 
model_list = []
for part in lbc:
    for traj in part.trajectories:
        if  traj.gradient > 2*np.sqrt(slope*traj.data.AF.iloc[0]) and \
            traj.data.AF.mean() < 0.4 and \
           len(traj.data.AF)>2:
        
            model_list.append(modelling.model_emcee(x=traj.data.wave.tolist(),
                                                    y=traj.data.AF.tolist(),
                                                    mutation=traj.mutation,
                                                    gene=traj.mutation.split()[0],
                                                    id=part.id))

In [43]:
# compute the trajectory simulations in parallel for all mutations 
pool = mp.Pool()
model_list = list(pool.map(modelling.logistic_emcee, model_list))

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars



The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [ 65.04358127  81.54503292 106.35251583  91.57540593]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [80.6950782  66.46635919 57.71124874 88.49650044]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [71.15714928 53.22603645 87.27419202 75.07833597]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [ 79.81438732  50.17248998  58.12325129 104.78185308]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars



The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [71.71275304 53.08510806 53.43982666 80.3838857 ]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [48.19406818 46.42003933 50.1155972  70.31298957]


/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars



...

The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [78.6215113  56.76105278 56.86411826 91.46304938]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [69.96089685 59.10737559 60.92105652 75.12435796]


/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars



The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [68.6752037  70.15249937 60.1440785  73.32255529]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [65.13672291 60.09561368 56.08800384 78.51589369]


/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning:

invalid value encountered in double_scalars



The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [71.11570775 54.3977616  53.92307758 78.21716379]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [76.27974396 65.71815538 49.8933121  94.19209102]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [69.55201154 69.75202135 54.3271817  74.39499233]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 20;
tau: [72.38685101 52.84178966 53.67678931 88.52405477]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 =

In [44]:
import pickle
filename = 'fitted_trajectories.pkl'
outfile = open(filename,'wb')
pickle.dump(model_list,outfile)
outfile.close()